In [0]:
#!rm -r ./save
!unzip ./save.zip
!rm -r ./VAR-DATASET

!wget --output-document=model.py https://juanvar.ams3.digitaloceanspaces.com/model.py 
!wget --output-document=driving_data.py https://juanvar.ams3.digitaloceanspaces.com/driving_data.py
!wget --output-document=VAR-DATASET.zip https://juanvar.ams3.digitaloceanspaces.com/VAR-DATASET.zip
#!wget --output-document=savePesosGit.zip  https://juanvar.ams3.digitaloceanspaces.com/savePesosGit.zip
!unzip VAR-DATASET.zip
#!unzip savePesosGit.zip


# New Section

In [0]:
!cat VAR-DATASET/data.txt

In [0]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

LOGDIR = './save'

#sess = tf.InteractiveSession()
#saver = tf.train.Saver()
#saver.restore(sess, "save/model.ckpt")


sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 60
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch: 0, Step: 0, Loss: 6.99019
Epoch: 0, Step: 10, Loss: 6.13497
Epoch: 0, Step: 20, Loss: 6.09051
Epoch: 0, Step: 30, Loss: 6.02032
Epoch: 0, Step: 40, Loss: 5.95525
Epoch: 0, Step: 50, Loss: 5.89527
Epoch: 0, Step: 60, Loss: 5.82713
Epoch: 0, Step: 70, Loss: 5.765
Model saved in file: ./save/model.ckpt
Epoch: 1, Step: 100, Loss: 5.75481
Epoch: 1, Step: 110, Loss: 5.69307
Epoch: 1, Step: 120, Loss: 5.63576
Epoch: 1, Step: 130, Loss: 5.57549
Epoch: 1, Step: 140, Loss: 5.51737
Epoch: 1, Step: 150, Loss: 5.46201
Epoch: 1, Step: 160, Loss: 5.40518
Epoch: 1, Step: 170, Loss: 5.34957
Model saved in file: ./save/model.ckpt
Epoch: 2, Step: 200, Loss: 5.33787
Epoch: 2, Step: 210, Loss: 5.28476
Epoch: 2, Step: 220, Loss: 5.23142
Epoch: 2, Step: 230, Loss: 5.17752
Epoch: 2, Step: 240, Loss: 5.12467
Epoch: 2, Step: 250, Loss: 5.07252
Epoch: 2, Step: 260, Loss: 5.02048
Epoch: 2, Step: 270, Loss: 4.96994
Model saved in file: ./save/model.ckpt
Epoch: 3, Step: 300, Loss: 4.95916
Epoch: 3, Step: 310

In [0]:
!zip -r save.zip ./save
!zip -r logs.zip ./logs

updating: save/ (stored 0%)
updating: save/model.ckpt.data-00000-of-00001 (deflated 42%)
updating: save/checkpoint (deflated 42%)
updating: save/model.ckpt.index (deflated 55%)
updating: save/model.ckpt.meta (deflated 89%)
updating: logs/ (stored 0%)
updating: logs/events.out.tfevents.1553625626.8afe43f460cd (deflated 84%)
  adding: logs/events.out.tfevents.1553627903.8afe43f460cd (deflated 84%)
